# 2차 라벨링
1. 백테스팅할때 방식처럼 1번 매수하면 그사이 매수는 무시하고 다음 sell로 넘어가는 1회,1회 매매하는대로 라벨링
2. 1일 뒤를 기준으로 라벨을 삼는 Future_label 생성

## Info
1. 백테스팅 호환 로직

- 1회 매수 → 1회 매도 패턴 강제
- 연속 신호 방지 (매수 후 또 매수 불가)
- 기존 백테스터와 100% 동일한 매매 로직

2. Future_Label 생성

- 1일 후 가격 기준으로 라벨 생성
- 2% 이상 상승 예상 → 강매수(2)
- 2% 이상 하락 예상 → 강매도(0)
- 그 외 → 홀드(1)

3. 암호화폐 특성 반영

- timezone 안전 처리
- 높은 변동성 고려한 임계값
- 라벨링된 CSV 파일 자동 감지

## 📊 생성되는 라벨들
- Optimized_Label: 백테스터 로직으로 정제된 라벨
- Future_Label: 1일 후 가격 변화 기준 라벨
- Optimized_Label_Name: 라벨명 (Strong_Sell/Hold/Strong_Buy)
- Future_Label_Name: Future 라벨명

In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import glob

def create_optimized_crypto_labels(df, window=1):
    """
    백테스터와 동일한 로직으로 암호화폐 라벨 생성 + 타이밍 최적화
    
    Parameters:
    -----------
    df : pd.DataFrame
        원본 데이터 (Close, Label 컬럼 필요)
    window : int
        타이밍 최적화 윈도우 (앞뒤 며칠)
    
    Returns:
    --------
    pd.DataFrame : 최적화된 라벨이 추가된 데이터
    """
    
    result_df = df.copy()
    n = len(df)
    
    # 원본 라벨
    original_labels = df['Label'].values
    
    # 1단계: 백테스터와 동일한 로직으로 유효한 신호만 추출
    valid_signals = []  # (index, signal_type) 형태로 저장
    current_position = 'CASH'  # CASH 또는 LONG
    
    print(f"   백테스터 로직으로 유효 신호 추출 중...")
    
    for i in range(n):
        current_label = original_labels[i]
        
        # 백테스터와 동일한 매매 조건
        if current_label == 2 and current_position == 'CASH':  # Buy 가능
            valid_signals.append((i, 'BUY'))
            current_position = 'LONG'
            
        elif current_label == 0 and current_position == 'LONG':  # Sell 가능
            valid_signals.append((i, 'SELL'))
            current_position = 'CASH'
    
    buy_signals = [idx for idx, signal in valid_signals if signal == 'BUY']
    sell_signals = [idx for idx, signal in valid_signals if signal == 'SELL']
    
    print(f"   원본 신호: Buy {sum(original_labels == 2)}개, Sell {sum(original_labels == 0)}개")
    print(f"   유효 신호: Buy {len(buy_signals)}개, Sell {len(sell_signals)}개")
    
    # 2단계: 타이밍 최적화 (window=1이면 최적화 생략)
    if window > 1:
        optimized_buy_signals = []
        optimized_sell_signals = []
        
        # Buy 신호 최적화 (앞뒤 window일 중 가장 낮은 가격일)
        for buy_idx in buy_signals:
            start_idx = max(0, buy_idx - window)
            end_idx = min(n - 1, buy_idx + window)
            
            # 해당 구간의 종가들과 인덱스들
            window_section = df.iloc[start_idx:end_idx + 1]
            min_price_idx = window_section['Close'].idxmin()
            
            # DataFrame 인덱스를 정수 인덱스로 변환
            best_buy_idx = df.index.get_loc(min_price_idx)
            optimized_buy_signals.append(best_buy_idx)
        
        # Sell 신호 최적화 (앞뒤 window일 중 가장 높은 가격일)
        for sell_idx in sell_signals:
            start_idx = max(0, sell_idx - window)
            end_idx = min(n - 1, sell_idx + window)
            
            # 해당 구간의 종가들과 인덱스들
            window_section = df.iloc[start_idx:end_idx + 1]
            max_price_idx = window_section['Close'].idxmax()
            
            # DataFrame 인덱스를 정수 인덱스로 변환
            best_sell_idx = df.index.get_loc(max_price_idx)
            optimized_sell_signals.append(best_sell_idx)
    else:
        # window=1이면 최적화 없이 원본 사용
        optimized_buy_signals = buy_signals
        optimized_sell_signals = sell_signals
    
    # 3단계: 최적화된 라벨 생성
    optimized_labels = np.ones(n, dtype=int)  # 기본값: Hold(1)
    
    # Buy 신호 적용
    for buy_idx in optimized_buy_signals:
        if 0 <= buy_idx < n:
            optimized_labels[buy_idx] = 2  # Strong_Buy
    
    # Sell 신호 적용
    for sell_idx in optimized_sell_signals:
        if 0 <= sell_idx < n:
            optimized_labels[sell_idx] = 0  # Strong_Sell
    
    # 4단계: 최종 검증 (백테스터 로직 재확인)
    final_labels = np.ones(n, dtype=int)  # 기본값: Hold(1)
    current_position = 'CASH'
    
    for i in range(n):
        current_label = optimized_labels[i]
        
        if current_label == 2 and current_position == 'CASH':  # Buy 가능
            final_labels[i] = 2
            current_position = 'LONG'
            
        elif current_label == 0 and current_position == 'LONG':  # Sell 가능
            final_labels[i] = 0
            current_position = 'CASH'
        
        # 나머지는 Hold(1) 유지
    
    # 5단계: Future_Label 생성 (1일 후 가격 기준)
    future_labels = np.ones(n, dtype=int)  # 기본값: Hold(1)
    
    for i in range(n-1):  # 마지막 날은 제외
        current_price = df.iloc[i]['Close']
        next_price = df.iloc[i+1]['Close']
        
        # 1일 후 가격 변화율
        price_change = (next_price - current_price) / current_price
        
        # 임계값 설정 (암호화폐 특성상 변동성이 크므로 조정 가능)
        buy_threshold = 0.02   # 2% 이상 상승 예상 시 매수
        sell_threshold = -0.02  # 2% 이상 하락 예상 시 매도
        
        if price_change >= buy_threshold:
            future_labels[i] = 2  # Strong_Buy
        elif price_change <= sell_threshold:
            future_labels[i] = 0  # Strong_Sell
        # else: Hold(1) 유지
    
    # 6단계: Future_Label도 백테스터 로직으로 검증
    final_future_labels = np.ones(n, dtype=int)
    current_position = 'CASH'
    
    for i in range(n):
        current_label = future_labels[i]
        
        if current_label == 2 and current_position == 'CASH':
            final_future_labels[i] = 2
            current_position = 'LONG'
        elif current_label == 0 and current_position == 'LONG':
            final_future_labels[i] = 0
            current_position = 'CASH'
    
    # 결과 저장
    result_df['Optimized_Label'] = final_labels
    result_df['Future_Label'] = final_future_labels
    
    # 라벨명 추가
    label_names = {0: 'Strong_Sell', 1: 'Hold', 2: 'Strong_Buy'}
    result_df['Optimized_Label_Name'] = result_df['Optimized_Label'].map(label_names)
    result_df['Future_Label_Name'] = result_df['Future_Label'].map(label_names)
    
    # 최종 통계
    final_buy_count = sum(final_labels == 2)
    final_sell_count = sum(final_labels == 0)
    final_hold_count = sum(final_labels == 1)
    
    future_buy_count = sum(final_future_labels == 2)
    future_sell_count = sum(final_future_labels == 0)
    future_hold_count = sum(final_future_labels == 1)
    
    print(f"   최종 라벨: Buy {final_buy_count}개, Sell {final_sell_count}개, Hold {final_hold_count}개")
    print(f"   Future 라벨: Buy {future_buy_count}개, Sell {future_sell_count}개, Hold {future_hold_count}개")
    print(f"   Buy-Sell 균형: {min(final_buy_count, final_sell_count)}/{max(final_buy_count, final_sell_count)}")
    
    return result_df

def process_crypto_labels_optimization(input_path, output_path, window=1, test_symbols=None):
    """
    암호화폐 라벨링 CSV 파일들을 읽어서 최적화된 라벨을 적용
    
    Parameters:
    -----------
    input_path : str
        입력 CSV 파일들 경로 (라벨링된 파일들)
    output_path : str
        출력 경로
    window : int
        타이밍 최적화 윈도우 (1=최적화없음, 3=앞뒤3일)
    test_symbols : list
        특정 심볼만 처리 (None이면 전체)
    
    Returns:
    --------
    dict : 최적화된 데이터들
    """
    
    print(f"🚀 암호화폐 라벨 최적화 시작!")
    print(f"📁 입력 경로: {input_path}")
    print(f"📁 출력 경로: {output_path}")
    print(f"🔧 최적화 윈도우: {window}일 ({'최적화 없음' if window == 1 else f'앞뒤 {window}일'})")
    print("="*80)
    
    # 라벨링된 CSV 파일 찾기
    patterns = [
        "*_labeled_dataset.csv",  # A00_00에서 생성된 파일
        "*_labeled.csv",
        "*.csv"
    ]
    
    csv_files = []
    used_pattern = None
    
    for pattern in patterns:
        files = glob.glob(os.path.join(input_path, pattern))
        if files:
            csv_files = files
            used_pattern = pattern
            print(f"✅ 파일 패턴 발견: {pattern} ({len(files)}개 파일)")
            break
    
    if not csv_files:
        print(f"❌ CSV 파일을 찾을 수 없습니다: {input_path}")
        return None
    
    # 출력 경로 생성
    os.makedirs(output_path, exist_ok=True)
    print(f"📂 출력 디렉토리 생성: {output_path}")
    
    # 데이터 로딩 및 처리
    optimized_cryptos = {}
    
    print(f"\n📊 암호화폐 데이터 처리 중...")
    
    for file_path in tqdm(csv_files, desc="파일 처리"):
        filename = os.path.basename(file_path)
        
        # 심볼명 추출
        symbol = filename.replace('.csv', '')
        for suffix in ['_labeled_dataset', '_labeled', '_enhanced', '_processed']:
            symbol = symbol.replace(suffix, '')
        
        # 특정 심볼만 처리
        if test_symbols and symbol not in test_symbols:
            continue
        
        try:
            print(f"\n🔍 처리 중: {symbol}")
            
            # CSV 읽기
            df = pd.read_csv(file_path)
            
            if df.empty:
                print(f"   ⚠️ 빈 데이터")
                continue
            
            # 날짜 인덱스 설정
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                if df['Date'].dt.tz is not None:
                    df['Date'] = df['Date'].dt.tz_localize(None)
                df = df.set_index('Date')
            
            # 필수 컬럼 확인
            required_cols = ['Close', 'Label']
            missing_cols = [col for col in required_cols if col not in df.columns]
            
            if missing_cols:
                print(f"   ⚠️ 필수 컬럼 누락: {missing_cols}")
                continue
            
            print(f"   📏 데이터 크기: {df.shape}")
            
            # 라벨 분포 확인
            label_dist = df['Label'].value_counts().sort_index()
            print(f"   🏷️ 원본 라벨 분포: {dict(label_dist)}")
            
            # 최적화된 라벨 생성
            optimized_df = create_optimized_crypto_labels(df, window=window)
            
            # 결과 저장
            optimized_cryptos[symbol] = optimized_df
            
            # CSV 파일로 저장
            output_filename = f"{symbol}_optimized.csv"
            output_file_path = os.path.join(output_path, output_filename)
            optimized_df.to_csv(output_file_path)
            
            print(f"   ✅ 저장 완료: {output_filename}")
            
        except Exception as e:
            print(f"   ❌ 오류 발생: {e}")
            continue
    
    # 최종 결과 요약
    print(f"\n" + "="*80)
    print(f"🎯 암호화폐 라벨 최적화 완료!")
    print(f"   성공: {len(optimized_cryptos)}개 암호화폐")
    print(f"   저장 경로: {output_path}")
    print(f"   파일 형식: [심볼명]_optimized.csv")
    
    # 각 심볼별 최종 통계
    print(f"\n📊 심볼별 라벨 통계:")
    print(f"{'심볼':<8} {'원본 Buy':<10} {'최적 Buy':<10} {'Future Buy':<12} {'원본 Sell':<11} {'최적 Sell':<11} {'Future Sell'}")
    print("-" * 85)
    
    for symbol, df in optimized_cryptos.items():
        orig_buy = sum(df['Label'] == 2)
        opt_buy = sum(df['Optimized_Label'] == 2)
        fut_buy = sum(df['Future_Label'] == 2)
        orig_sell = sum(df['Label'] == 0)
        opt_sell = sum(df['Optimized_Label'] == 0)
        fut_sell = sum(df['Future_Label'] == 0)
        
        print(f"{symbol:<8} {orig_buy:<10} {opt_buy:<10} {fut_buy:<12} {orig_sell:<11} {opt_sell:<11} {fut_sell}")
    
    return optimized_cryptos

def compare_crypto_label_performance(df, original_col='Label', optimized_col='Optimized_Label', future_col='Future_Label'):
    """
    암호화폐 라벨 성능 비교 분석
    
    Parameters:
    -----------
    df : pd.DataFrame
        세 종류의 라벨이 모두 있는 데이터
    """
    
    print("\n📊 라벨 성능 비교 분석")
    print("="*60)
    
    # 라벨 분포 비교
    original_dist = df[original_col].value_counts().sort_index()
    optimized_dist = df[optimized_col].value_counts().sort_index()
    future_dist = df[future_col].value_counts().sort_index()
    
    print("라벨 분포 비교:")
    print(f"{'라벨':<15} {'원본':<8} {'최적화':<8} {'Future':<8}")
    print("-" * 45)
    
    label_names = {0: 'Strong_Sell', 1: 'Hold', 2: 'Strong_Buy'}
    for label in [0, 1, 2]:
        orig_count = original_dist.get(label, 0)
        opt_count = optimized_dist.get(label, 0)
        fut_count = future_dist.get(label, 0)
        
        print(f"{label_names[label]:<15} {orig_count:<8} {opt_count:<8} {fut_count:<8}")
    
    # 매매 패턴 분석
    def analyze_trading_pattern(labels):
        complete_trades = 0
        hold_periods = []
        current_position = 'CASH'
        last_buy_idx = None
        
        for i, label in enumerate(labels):
            if label == 2 and current_position == 'CASH':  # Buy
                last_buy_idx = i
                current_position = 'LONG'
            elif label == 0 and current_position == 'LONG':  # Sell
                if last_buy_idx is not None:
                    hold_period = i - last_buy_idx
                    hold_periods.append(hold_period)
                    complete_trades += 1
                current_position = 'CASH'
        
        avg_hold_period = np.mean(hold_periods) if hold_periods else 0
        
        return {
            'complete_trades': complete_trades,
            'avg_hold_period': avg_hold_period,
            'hold_periods': hold_periods
        }
    
    # 각 라벨의 매매 패턴 분석
    original_pattern = analyze_trading_pattern(df[original_col].values)
    optimized_pattern = analyze_trading_pattern(df[optimized_col].values)
    future_pattern = analyze_trading_pattern(df[future_col].values)
    
    print(f"\n매매 패턴 분석:")
    print(f"{'지표':<20} {'원본':<12} {'최적화':<12} {'Future':<12}")
    print("-" * 60)
    print(f"{'완전 매매 횟수':<20} {original_pattern['complete_trades']:<12} {optimized_pattern['complete_trades']:<12} {future_pattern['complete_trades']:<12}")
    print(f"{'평균 보유기간':<20} {original_pattern['avg_hold_period']:.1f}일{'':<6} {optimized_pattern['avg_hold_period']:.1f}일{'':<6} {future_pattern['avg_hold_period']:.1f}일")
    
    return {
        'distributions': {
            'original': original_dist.to_dict(),
            'optimized': optimized_dist.to_dict(),
            'future': future_dist.to_dict()
        },
        'patterns': {
            'original': original_pattern,
            'optimized': optimized_pattern,
            'future': future_pattern
        }
    }

# 메인 실행 함수 (기존 함수명과 호환)
def process_stocks_with_optimized_labels(input_path, output_path, window=1, test_symbols=None):
    """암호화폐 라벨 최적화 메인 함수"""
    return process_crypto_labels_optimization(input_path, output_path, window, test_symbols)



# 실행

In [12]:
# 사용 예시
if __name__ == "__main__":
    # 예시 경로
    input_path = "/workspace/AI모델/projects/coin/data/v01/labeled"
    output_path = "/workspace/AI모델/projects/coin/data/v01/optimized"
    
    # 전체 종목 처리
    optimized_cryptos = process_stocks_with_optimized_labels(
        input_path=input_path,
        output_path=output_path,
        window=1  # 1일 윈도우 (최적화 없음, 백테스터 로직만 적용)
    )
    
    # 결과가 있으면 성능 비교
    if optimized_cryptos:
        # 첫 번째 심볼로 성능 비교 예시
        first_symbol = list(optimized_cryptos.keys())[0]
        compare_crypto_label_performance(optimized_cryptos[first_symbol])

🚀 암호화폐 라벨 최적화 시작!
📁 입력 경로: /workspace/AI모델/projects/coin/data/v01/labeled
📁 출력 경로: /workspace/AI모델/projects/coin/data/v01/optimized
🔧 최적화 윈도우: 1일 (최적화 없음)
✅ 파일 패턴 발견: *_labeled_dataset.csv (50개 파일)
📂 출력 디렉토리 생성: /workspace/AI모델/projects/coin/data/v01/optimized

📊 암호화폐 데이터 처리 중...


파일 처리:   0%|          | 0/50 [00:00<?, ?it/s]


🔍 처리 중: AAVE
   📏 데이터 크기: (1821, 54)
   🏷️ 원본 라벨 분포: {0: 820, 1: 182, 2: 819}
   백테스터 로직으로 유효 신호 추출 중...
   원본 신호: Buy 819개, Sell 820개
   유효 신호: Buy 130개, Sell 130개


파일 처리:   2%|▏         | 1/50 [00:00<00:18,  2.67it/s]

   최종 라벨: Buy 130개, Sell 130개, Hold 1561개
   Future 라벨: Buy 326개, Sell 325개, Hold 1170개
   Buy-Sell 균형: 130/130
   ✅ 저장 완료: AAVE_optimized.csv

🔍 처리 중: ADA
   📏 데이터 크기: (2879, 54)
   🏷️ 원본 라벨 분포: {0: 1296, 1: 287, 2: 1296}
   백테스터 로직으로 유효 신호 추출 중...
   원본 신호: Buy 1296개, Sell 1296개
   유효 신호: Buy 173개, Sell 173개


...

   최종 라벨: Buy 198개, Sell 197개, Hold 2484개
   Future 라벨: Buy 405개, Sell 404개, Hold 2070개
   Buy-Sell 균형: 197/198
   ✅ 저장 완료: XLM_optimized.csv

🔍 처리 중: XMR
   📏 데이터 크기: (2879, 54)
   🏷️ 원본 라벨 분포: {0: 1296, 1: 287, 2: 1296}
   백테스터 로직으로 유효 신호 추출 중...
   원본 신호: Buy 1296개, Sell 1296개
   유효 신호: Buy 254개, Sell 254개


파일 처리:  98%|█████████▊| 49/50 [00:18<00:00,  2.25it/s]

   최종 라벨: Buy 254개, Sell 254개, Hold 2371개
   Future 라벨: Buy 417개, Sell 417개, Hold 2045개
   Buy-Sell 균형: 254/254
   ✅ 저장 완료: XMR_optimized.csv

🔍 처리 중: XRP
   📏 데이터 크기: (2879, 54)
   🏷️ 원본 라벨 분포: {0: 1296, 1: 287, 2: 1296}
   백테스터 로직으로 유효 신호 추출 중...
   원본 신호: Buy 1296개, Sell 1296개
   유효 신호: Buy 203개, Sell 203개


파일 처리: 100%|██████████| 50/50 [00:18<00:00,  2.68it/s]

   최종 라벨: Buy 203개, Sell 203개, Hold 2473개
   Future 라벨: Buy 373개, Sell 373개, Hold 2133개
   Buy-Sell 균형: 203/203
   ✅ 저장 완료: XRP_optimized.csv

🎯 암호화폐 라벨 최적화 완료!
   성공: 50개 암호화폐
   저장 경로: /workspace/AI모델/projects/coin/data/v01/optimized
   파일 형식: [심볼명]_optimized.csv

📊 심볼별 라벨 통계:
심볼       원본 Buy     최적 Buy     Future Buy   원본 Sell     최적 Sell     Future Sell
-------------------------------------------------------------------------------------
AAVE     819        130        326          820         130         325
ADA      1296       173        434          1296        173         433
AETHWETH 62         12         15           62          12          14
ALGO     1031       156        393          1031        156         392
AVAX     825        128        304          825         128         304
BCH      1296       223        423          1296        223         422
BGB      685        103        142          685         103         142
BNB      1296       197        355          1296   